In [5]:
from dash import Dash, dcc, html, Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import dash_bootstrap_components as dbc
import dash_daq as daq

# ---------------------------------------------------------
# App mit Bootstrap-Theme
# ---------------------------------------------------------
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# ---------------------------------------------------------
# DATEN LADEN
# ---------------------------------------------------------

# 1) Linechart-Datenbasis – Kinosäle pro Jahr pro Kanton (für Sprachraum-Auswertung)
df_raw_line = pd.read_csv("Kinosäle_legende.csv", sep=";")

# Jahres-Spalten definieren
year_cols = [str(j) for j in range(2014, 2025)]

# Robust: Kantonsspalte finden (alles, was NICHT Jahr ist)
non_year_cols = [c for c in df_raw_line.columns if c not in year_cols]
kanton_col = non_year_cols[0] if len(non_year_cols) > 0 else None

if kanton_col is None:
    raise ValueError("Kinosäle_legende.csv: Konnte keine Kantonsspalte finden (nur Jahr-Spalten erkannt).")

df_raw_line = df_raw_line.rename(columns={kanton_col: "Kanton"})

# Gesamtschweiz (dein bisheriges Linechart)
totals = df_raw_line[year_cols].sum()
df_line_total = totals.reset_index()
df_line_total.columns = ["Year", "Cinema_Rooms"]
df_line_total["Year"] = df_line_total["Year"].astype(int)
df_line_total["Cinema_Rooms"] = df_line_total["Cinema_Rooms"] / 2  # wie bisher

# --- NEU: Sprachraum-Zuordnung nach eurer Einteilung ---
# Wir unterstützen sowohl Abkürzungen (VD) als auch deutsche Kantonsnamen (Waadt).
romandie_set = {
    "VD", "NE", "GE", "JU",
    "WAADT", "NEUENBURG", "GENF", "JURA",
    "VAUD", "NEUCHATEL", "GENEVE", "GENÈVE"
}
bilingual_set = {
    "VS", "GR", "FR", "BE",
    "WALLIS", "GRAUBÜNDEN", "GRAUBUENDEN", "FREIBURG", "BERN",
    "VALAIS", "GRISONS", "FRIBOURG", "BERNE"
}
ticino_set = {
    "TI", "TESSIN", "TICINO"
}

def normalize_canton(x):
    if pd.isna(x):
        return ""
    s = str(x).strip()
    return s.upper()

def map_sprachraum(kanton_value):
    k = normalize_canton(kanton_value)
    if k in romandie_set:
        return "Romandie"
    if k in bilingual_set:
        return "Mehrsprachig"
    if k in ticino_set:
        return "Tessin"
    return "Deutschschweiz"

df_rooms_long = df_raw_line.melt(
    id_vars=["Kanton"],
    value_vars=year_cols,
    var_name="Year",
    value_name="Cinema_Rooms"
)
df_rooms_long["Year"] = df_rooms_long["Year"].astype(int)
df_rooms_long["Cinema_Rooms"] = pd.to_numeric(df_rooms_long["Cinema_Rooms"], errors="coerce")
df_rooms_long["Sprachraum"] = df_rooms_long["Kanton"].apply(map_sprachraum)

# Sprachraum-Summen pro Jahr
df_language = (
    df_rooms_long
    .groupby(["Year", "Sprachraum"], as_index=False)["Cinema_Rooms"]
    .sum()
)
df_language["Cinema_Rooms"] = df_language["Cinema_Rooms"] / 2  # konsistent mit deinem bisherigen /2

sprachraum_options = ["Deutschschweiz", "Romandie", "Mehrsprachig", "Tessin"]
sprachraum_default = ["Deutschschweiz", "Romandie"]  # Fokus Hypothese

sprachraum_colors = {
    "Deutschschweiz": "#1F4E79",
    "Romandie": "#C00000",
    "Mehrsprachig": "#76A5AF",
    "Tessin": "#38761D",
}

# 2) Columnchart – Kantonskategorien
years = ['2014', '2015', '2016', '2017', '2018',
         '2019', '2020', '2021', '2022', '2023', '2024']

staedtische = [160, 159, 162, 159, 162, 155, 152, 146, 149, 147, 143]
urbane      = [82, 81, 81, 78, 77, 78, 76, 76, 75, 75, 75]
laendliche  = [33, 33, 35, 34, 37, 36, 35, 35, 38, 38, 37]

df_bar = pd.DataFrame({
    "Jahr": years,
    "Städtische Kantone": staedtische,
    "Urbane Kantone": urbane,
    "Ländliche Kantone": laendliche
})

df_bar_long = df_bar.melt(
    id_vars="Jahr",
    var_name="Kategorie",
    value_name="Kinoanzahl"
)

category_labels = {
    "Städtische Kantone": "Städtische Kantone 🏙️",
    "Urbane Kantone": "Urbane Kantone 🏘️",
    "Ländliche Kantone": "Ländliche Kantone 🌿",
}
df_bar_long["Kategorie"] = df_bar_long["Kategorie"].replace(category_labels)

bar_colors = {
    "Städtische Kantone 🏙️": "#9E9E9E",
    "Urbane Kantone 🏘️": "#D8C3A5",
    "Ländliche Kantone 🌿": "#2E7D32",
}

# 3) Donutchart + Datenbasis für "pro Kanton"-Säulendiagramm
df_pie = pd.read_csv("Kinos_legende.csv", sep=";")
pie_years = [col for col in df_pie.columns if col.isdigit()]
start_year = pie_years[-1]  # z.B. 2024

df_canton = df_pie.copy()
df_canton = df_canton[df_canton["Kantone"] != "Total"]

df_canton_long = df_canton.melt(
    id_vars="Kantone",
    value_vars=pie_years,
    var_name="Jahr",
    value_name="Kinoanzahl"
)
df_canton_long["Jahr"] = df_canton_long["Jahr"].astype(int)
df_canton_long["Kinoanzahl"] = pd.to_numeric(df_canton_long["Kinoanzahl"], errors="coerce")

kantone_liste = sorted(df_canton_long["Kantone"].dropna().unique())
alphabet = px.colors.qualitative.Alphabet  # 26 Farben
kanton_colors = {k: alphabet[i % len(alphabet)] for i, k in enumerate(kantone_liste)}

last_year_int = int(start_year)
top8 = (
    df_canton_long[df_canton_long["Jahr"] == last_year_int]
    .sort_values("Kinoanzahl", ascending=False)
    .head(8)["Kantone"]
    .tolist()
)

# 4) Scatterplot – Kinos vs. Säle pro Kanton
df_scatter = pd.read_csv(
    "Kinos_vs_Saele_2024.csv",
    dtype={
        "Kanton": "string",
        "Kinos_2024": "float",
        "Saele_2024": "float"
    }
)
df_scatter = df_scatter.dropna(subset=["Kanton", "Kinos_2024", "Saele_2024"]).copy()

slider_min = int(df_scatter["Saele_2024"].min())
slider_max = int(df_scatter["Saele_2024"].max())

# ---------------------------------------------------------
# LAYOUT – Tabs
# ---------------------------------------------------------

app.layout = dbc.Container(
    [
        html.H1("Kino-Dashboard Schweiz", className="my-4"),

        dbc.Tabs(
            [
                dbc.Tab(
                    label="Linechart",
                    children=[
                        html.Br(),

                        daq.ToggleSwitch(
                            id="line_mode_switch",
                            value=False,  # False = CH total, True = Sprachräume
                            label="CH total  /  Sprachräume",
                            labelPosition="top"
                        ),

                        # Controls: je nach Modus ein-/ausblenden
                        html.Div(
                            id="line_total_controls",
                            children=[
                                html.Br(),
                                html.Label("Linienfarbe:"),
                                dcc.Dropdown(
                                    options=['red', 'green', 'blue', 'orange', 'purple'],
                                    value='blue',
                                    id='line_color',
                                    clearable=False,
                                    style={"width": "250px"}
                                ),
                            ],
                        ),

                        html.Div(
                            id="line_language_controls",
                            children=[
                                html.Br(),
                                html.Label("Sprachräume auswählen:"),
                                dcc.Dropdown(
                                    id="sprachraum_multi",
                                    options=[{"label": s, "value": s} for s in sprachraum_options],
                                    value=sprachraum_default,
                                    multi=True,
                                    clearable=False,
                                    style={"maxWidth": "520px"}
                                ),
                            ],
                        ),

                        dcc.Graph(id="graph_line")
                    ],
                ),

                dbc.Tab(
                    label="Columnchart",
                    children=[
                        html.Br(),

                        daq.ToggleSwitch(
                            id="barmode-switch",
                            value=False,  # False = Kategorie, True = Kanton
                            label="Kantonskategorie  /  Kanton",
                            labelPosition="top"
                        ),

                        html.Div(
                            id="kanton_dropdown_wrapper",
                            children=[
                                html.Br(),
                                html.Label("Kantone auswählen:"),
                                dcc.Dropdown(
                                    id="kanton_multi",
                                    options=[{"label": k, "value": k} for k in kantone_liste],
                                    value=top8,
                                    multi=True,
                                    clearable=False,
                                    style={"maxWidth": "520px"}
                                ),

                                dbc.Button(
                                    "Alle Kantone anzeigen",
                                    id="toggle_kantone_btn",
                                    color="secondary",
                                    outline=True,
                                    className="mt-2"
                                ),

                                dcc.Store(id="show_all_kantone", data=False),
                            ],
                        ),

                        dcc.Graph(id="graph_bar")
                    ],
                ),

                dbc.Tab(
                    label="Donutchart",
                    children=[
                        html.Br(),
                        html.Label("Jahr auswählen:"),
                        dcc.Dropdown(
                            id="jahr-dropdown",
                            options=[{"label": j, "value": j} for j in pie_years],
                            value=start_year,
                            clearable=False,
                            style={"width": "200px", "marginBottom": "10px"}
                        ),
                        html.Label("Kanton auswählen:"),
                        dcc.RadioItems(
                            id="kanton-radio",
                            options=[],
                            value=None,
                            inline=True,
                            style={"marginBottom": "10px"},
                            labelStyle={"marginRight": "16px", "padding": "4px 0"},
                        ),
                        dcc.Graph(id="graph_pie")
                    ],
                ),

                dbc.Tab(
                    label="Scatterplot",
                    children=[
                        html.Br(),
                        html.Label("Minimale Anzahl Säle (Filter):"),
                        dcc.Slider(
                            id="saal_slider",
                            min=slider_min,
                            max=slider_max,
                            step=1,
                            value=slider_min,
                            marks={slider_min: str(slider_min), slider_max: str(slider_max)},
                            tooltip={"placement": "bottom", "always_visible": False}
                        ),
                        dcc.Graph(id="graph_scatter")
                    ],
                ),
            ]
        ),
    ],
    fluid=True,
)

# ---------------------------------------------------------
# CALLBACKS
# ---------------------------------------------------------

# Linechart: Controls ein-/ausblenden
@app.callback(
    Output("line_total_controls", "style"),
    Output("line_language_controls", "style"),
    Input("line_mode_switch", "value")
)
def toggle_line_controls(language_mode):
    if language_mode:
        return {"display": "none"}, {"display": "block"}
    return {"display": "block"}, {"display": "none"}


# 1) Linechart: Toggle zwischen CH total und Sprachräume
@app.callback(
    Output("graph_line", "figure"),
    Input("line_mode_switch", "value"),
    Input("line_color", "value"),
    Input("sprachraum_multi", "value"),
)
def update_line_chart(language_mode, color_value, selected_sprachraeume):

    # A) CH total (dein bisheriges Diagramm)
    if not language_mode:
        fig = px.line(
            df_line_total,
            x="Year",
            y="Cinema_Rooms",
            markers=True,
            color_discrete_sequence=[color_value]
        )
        fig.update_traces(line=dict(width=3), marker=dict(size=8))
        fig.update_layout(
            title="Entwicklung der Anzahl Kinosäle (2014–2024) – Gesamtschweiz",
            xaxis_title="Jahr",
            yaxis_title="Kinosäle",
            hovermode="x unified",
            template="plotly_white"
        )
        return fig

    # B) Sprachräume
    if not selected_sprachraeume:
        selected_sprachraeume = sprachraum_default

    dff = df_language[df_language["Sprachraum"].isin(selected_sprachraeume)].copy()

    fig = px.line(
        dff,
        x="Year",
        y="Cinema_Rooms",
        color="Sprachraum",
        markers=True,
        color_discrete_map=sprachraum_colors
    )
    fig.update_traces(line=dict(width=3), marker=dict(size=7))
    fig.update_layout(
        title="Entwicklung der Anzahl Kinosäle (2014–2024) nach Sprachraum",
        xaxis_title="Jahr",
        yaxis_title="Kinosäle",
        hovermode="x unified",
        template="plotly_white"
    )
    return fig


# Dropdown/Buttons nur anzeigen, wenn Toggle = Kanton
@app.callback(
    Output("kanton_dropdown_wrapper", "style"),
    Input("barmode-switch", "value")
)
def show_hide_kanton_dropdown(show_by_kanton):
    return {"display": "block"} if show_by_kanton else {"display": "none"}


# Button schaltet zwischen Top8 und Alle Kantone
@app.callback(
    Output("kanton_multi", "value"),
    Output("toggle_kantone_btn", "children"),
    Output("show_all_kantone", "data"),
    Input("toggle_kantone_btn", "n_clicks"),
    State("show_all_kantone", "data"),
    prevent_initial_call=True
)
def toggle_kantone(n_clicks, show_all):
    if show_all:
        return top8, "Alle Kantone anzeigen", False
    else:
        return kantone_liste, "Zurück zu Top 8", True


# 2) Columnchart – Toggle schaltet zwischen 2 Diagrammen
@app.callback(
    Output("graph_bar", "figure"),
    Input("barmode-switch", "value"),
    Input("kanton_multi", "value"),
)
def update_bar_chart(show_by_kanton, selected_kantone):

    # A) Kantonskategorie
    if not show_by_kanton:
        fig = px.bar(
            df_bar_long,
            x="Jahr",
            y="Kinoanzahl",
            color="Kategorie",
            barmode="group",
            color_discrete_map=bar_colors,
            category_orders={"Kategorie": list(bar_colors.keys())}
        )
        fig.update_layout(
            title="Kinoanzahl 2014–2024 nach Kantonskategorie",
            xaxis_title="Jahr",
            yaxis_title="Anzahl Kinos",
            legend_title="Kategorie",
            template="plotly_white"
        )
        return fig

    # B) Nach Kanton
    if not selected_kantone:
        selected_kantone = top8

    dff = df_canton_long[df_canton_long["Kantone"].isin(selected_kantone)].copy()

    fig = px.bar(
        dff,
        x="Jahr",
        y="Kinoanzahl",
        color="Kantone",
        barmode="group",
        color_discrete_map=kanton_colors
    )
    fig.update_layout(
        title="Kinoanzahl pro Jahr nach Kanton (Auswahl)",
        xaxis_title="Jahr",
        yaxis_title="Kinoanzahl",
        legend_title="Kanton",
        template="plotly_white"
    )
    return fig


# 3a) Donutchart – Kantonsliste aktualisieren
@app.callback(
    Output("kanton-radio", "options"),
    Output("kanton-radio", "value"),
    Input("jahr-dropdown", "value")
)
def update_kanton_options(selected_year):
    d = df_pie[["Kantone", selected_year]].dropna()
    d = d[d["Kantone"] != "Total"]

    kantone = d["Kantone"].tolist()
    first = kantone[0] if kantone else None

    options = [{"label": k, "value": k} for k in kantone]
    return options, first


# 3b) Donutchart – Figur (heller Donut + dunkler Hintergrund)
@app.callback(
    Output("graph_pie", "figure"),
    Input("jahr-dropdown", "value"),
    Input("kanton-radio", "value"),
)
def update_pie(selected_year, selected_kanton):
    if selected_kanton is None:
        return go.Figure()

    d = df_pie[["Kantone", selected_year]].dropna()
    d = d[d["Kantone"] != "Total"]

    labels = d["Kantone"].tolist()
    values = d[selected_year].astype(float).tolist()

    idx = labels.index(selected_kanton)

    pull_values = [0] * len(labels)
    pull_values[idx] = 0.12

    colors = ["aliceblue"] * len(labels)
    colors[idx] = "mediumorchid"

    fig = go.Figure(
        data=[
            go.Pie(
                labels=labels,
                values=values,
                pull=pull_values,
                marker=dict(colors=colors),
                textinfo="none",
                hole=0.45,
                hovertemplate="%{label}<br>%{value} Kinos<extra></extra>",
            )
        ]
    )

    center_text = f"{selected_kanton}<br><b>{values[idx]:.0f} Kinos</b>"

    dark_bg = "#111111"
    light_text = "#F2F2F2"

    fig.update_layout(
        title=dict(
            text=f"Kinos pro Kanton ({selected_year}) – Fokus: {selected_kanton}",
            font=dict(color=light_text)
        ),
        annotations=[
            dict(
                text=center_text,
                x=0.5,
                y=0.5,
                showarrow=False,
                font=dict(size=18, color=light_text)
            )
        ],
        showlegend=False,
        margin=dict(l=40, r=40, t=80, b=40),
        height=500,
        paper_bgcolor=dark_bg,
        plot_bgcolor=dark_bg,
        font=dict(color=light_text)
    )

    return fig


# 4) Scatterplot – Slider-Filter
@app.callback(
    Output("graph_scatter", "figure"),
    Input("saal_slider", "value")
)
def update_scatter(min_saal):
    d = df_scatter[df_scatter["Saele_2024"] >= min_saal]

    fig = px.scatter(
        d,
        x="Kinos_2024",
        y="Saele_2024",
        text="Kanton",
        color_discrete_sequence=["green"]
    )
    fig.update_traces(
        textposition="top center",
        mode="markers+text",
        marker=dict(size=12)
    )
    fig.update_layout(
        title=f"Kinos vs. Kinosäle pro Kanton (2024) – Filter: mind. {min_saal} Säle",
        xaxis_title="Kinos_2024",
        yaxis_title="Saele_2024",
        template="plotly_white"
    )
    return fig


# ---------------------------------------------------------
# START
# ---------------------------------------------------------
if __name__ == "__main__":
    app.run(debug=True, port=8985, jupyter_mode="external")


Dash app running on http://127.0.0.1:8985/
